# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Assignment - real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

### First I'll download all the data and set a few parameters for Pandas. 

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)  # Unlimited columns

In [2]:
#!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
#!unzip fma_metadata.zip


### Now I'll open the Tracks.csv and create my dataframe.


In [3]:
# Reading this CSV but the names of the columns look all messed up.
names = pd.read_csv('fma_metadata/tracks.csv')
names.head()

,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,album.9,album.10,album.11,album.12,artist,artist.1,artist.2,artist.3,artist.4,artist.5,artist.6,artist.7,artist.8,artist.9,artist.10,artist.11,artist.12,artist.13,artist.14,artist.15,artist.16,set,set.1,track,track.1,track.2,track.3,track.4,track.5,track.6,track.7,track.8,track.9,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments,date_created,favorites,id,latitude,location,longitude,members,name,related_projects,tags,website,wikipedia_page,split,subset,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:12,2008-11-26 00:00:00,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,medium,256000,0,NaN,2008-11-26 01:48:14,2008-11-26 00:00:00,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
4,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:20,2008-11-26 00:00:00,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World


Ooh, those headers look all messed up. I'll replace them with a cleaner list.

### Replace headers with tidy list of good names.

In [4]:
# So I need to make a list of the good names, and drop all the rows that aren't good. 
# I'm going to save these names and use them as the feature names.
cols = names[0:1].values.tolist()
cols = cols[0]
cols[0] = 'track_id'
print(f"The fixed list of names now: {cols[0:3]} ...\n")

# Instead of renaming these I'm going to reimport my csv and drop the top rows.
# This way it will import the datatypes for each column
tracks = pd.read_csv('fma_metadata/tracks.csv', skiprows=[0,1,2], header=None, names=cols)
tracks.head(2)

The fixed list of names now: ['track_id', 'comments', 'date_created'] ...



,track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments.1,date_created.1,favorites.1,id.1,latitude,location,longitude,members,name,related_projects,tags.1,website,wikipedia_page,split,subset,bit_rate,comments.2,composer,date_created.2,date_recorded,duration,favorites.2,genre_top,genres,genres_all,information.1,interest,language_code,license,listens.1,lyricist,number,publisher,tags.2,title.1
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:12,2008-11-26 00:00:00,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
1,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,medium,256000,0,NaN,2008-11-26 01:48:14,2008-11-26 00:00:00,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave


### Inspect the data a bit more....

In [5]:
print(tracks.shape)


(106574, 53)


### Here I'm going to add in the Features.csv field. 

These features are from the features extraction from the audio (used to create features.csv).
https://github.com/mdeff/fma

In [6]:
features = pd.read_csv('fma_metadata/features.csv', skiprows=[1,2,3])
features = features.rename(columns={'feature': 'track_id'})

In [7]:
# Here I printed out a longer head table to make sure that the rows allign.
# I also print out the shape to make sure it looks right for mergeing the two dataframes. 
print(features.shape)
features.head(3)

(106574, 519)


,track_id,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,chroma_cens.9,chroma_cens.10,chroma_cens.11,chroma_cens.12,chroma_cens.13,chroma_cens.14,chroma_cens.15,chroma_cens.16,chroma_cens.17,chroma_cens.18,chroma_cens.19,chroma_cens.20,chroma_cens.21,chroma_cens.22,chroma_cens.23,chroma_cens.24,chroma_cens.25,chroma_cens.26,chroma_cens.27,chroma_cens.28,chroma_cens.29,chroma_cens.30,chroma_cens.31,chroma_cens.32,chroma_cens.33,chroma_cens.34,chroma_cens.35,chroma_cens.36,chroma_cens.37,chroma_cens.38,chroma_cens.39,chroma_cens.40,chroma_cens.41,chroma_cens.42,chroma_cens.43,chroma_cens.44,chroma_cens.45,chroma_cens.46,chroma_cens.47,chroma_cens.48,chroma_cens.49,chroma_cens.50,chroma_cens.51,chroma_cens.52,chroma_cens.53,chroma_cens.54,chroma_cens.55,chroma_cens.56,chroma_cens.57,chroma_cens.58,chroma_cens.59,chroma_cens.60,chroma_cens.61,chroma_cens.62,chroma_cens.63,chroma_cens.64,chroma_cens.65,chroma_cens.66,chroma_cens.67,chroma_cens.68,chroma_cens.69,chroma_cens.70,chroma_cens.71,chroma_cens.72,chroma_cens.73,chroma_cens.74,chroma_cens.75,chroma_cens.76,chroma_cens.77,chroma_cens.78,chroma_cens.79,chroma_cens.80,chroma_cens.81,chroma_cens.82,chroma_cens.83,chroma_cqt,chroma_cqt.1,chroma_cqt.2,chroma_cqt.3,chroma_cqt.4,chroma_cqt.5,chroma_cqt.6,chroma_cqt.7,chroma_cqt.8,chroma_cqt.9,chroma_cqt.10,chroma_cqt.11,chroma_cqt.12,chroma_cqt.13,chroma_cqt.14,chroma_cqt.15,chroma_cqt.16,chroma_cqt.17,chroma_cqt.18,chroma_cqt.19,chroma_cqt.20,chroma_cqt.21,chroma_cqt.22,chroma_cqt.23,chroma_cqt.24,chroma_cqt.25,chroma_cqt.26,chroma_cqt.27,chroma_cqt.28,chroma_cqt.29,chroma_cqt.30,chroma_cqt.31,chroma_cqt.32,chroma_cqt.33,chroma_cqt.34,chroma_cqt.35,chroma_cqt.36,chroma_cqt.37,chroma_cqt.38,chroma_cqt.39,chroma_cqt.40,chroma_cqt.41,chroma_cqt.42,chroma_cqt.43,chroma_cqt.44,chroma_cqt.45,chroma_cqt.46,chroma_cqt.47,chroma_cqt.48,chroma_cqt.49,chroma_cqt.50,chroma_cqt.51,chroma_cqt.52,chroma_cqt.53,chroma_cqt.54,chroma_cqt.55,chroma_cqt.56,chroma_cqt.57,chroma_cqt.58,chroma_cqt.59,chroma_cqt.60,chroma_cqt.61,chroma_cqt.62,chroma_cqt.63,chroma_cqt.64,chroma_cqt.65,chroma_cqt.66,chroma_cqt.67,chroma_cqt.68,chroma_cqt.69,chroma_cqt.70,chroma_cqt.71,chroma_cqt.72,chroma_cqt.73,chroma_cqt.74,chroma_cqt.75,chroma_cqt.76,chroma_cqt.77,chroma_cqt.78,chroma_cqt.79,chroma_cqt.80,chroma_cqt.81,chroma_cqt.82,chroma_cqt.83,chroma_stft,chroma_stft.1,chroma_stft.2,chroma_stft.3,chroma_stft.4,chroma_stft.5,chroma_stft.6,chroma_stft.7,chroma_stft.8,chroma_stft.9,chroma_stft.10,chroma_stft.11,chroma_stft.12,chroma_stft.13,chroma_stft.14,chroma_stft.15,chroma_stft.16,chroma_stft.17,chroma_stft.18,chroma_stft.19,chroma_stft.20,chroma_stft.21,chroma_stft.22,chroma_stft.23,chroma_stft.24,chroma_stft.25,chroma_stft.26,chroma_stft.27,chroma_stft.28,chroma_stft.29,chroma_stft.30,chroma_stft.31,chroma_stft.32,chroma_stft.33,chroma_stft.34,chroma_stft.35,chroma_stft.36,chroma_stft.37,chroma_stft.38,chroma_stft.39,chroma_stft.40,chroma_stft.41,chroma_stft.42,chroma_stft.43,chroma_stft.44,chroma_stft.45,chroma_stft.46,chroma_stft.47,chroma_stft.48,chroma_stft.49,chroma_stft.50,chroma_stft.51,chroma_stft.52,chroma_stft.53,chroma_stft.54,chroma_stft.55,chroma_stft.56,chroma_stft.57,chroma_stft.58,chroma_stft.59,chroma_stft.60,chroma_stft.61,chroma_stft.62,chroma_stft.63,chroma_stft.64,chroma_stft.65,chroma_stft.66,chroma_stft.67,chroma_stft.68,chroma_stft.69,chroma_stft.70,chroma_stft.71,chroma_stft.72,chroma_stft.73,chroma_stft.74,chroma_stft.75,chroma_stft.76,chroma_stft.77,chroma_stft.78,chroma_stft.79,chroma_stft.80,chroma_stft.81,chroma_stft.82,chroma_stft.83,mfcc,mfcc.1,mfcc.2,mfcc.3,mfcc.4,mfcc.5,mfcc.6,mfcc.7,mfcc.8,mfcc.9,mfcc.10,mfcc.11,mfcc.12,mfcc.13,mfcc.14,mfcc.15,mfcc.16,mfcc.17,mfcc.18,mfcc.19,mfcc.20,mfcc.21,mfcc.22,mfcc.23,mfcc.24,mfcc.25,mfcc.26,mfcc.27,mfcc.28,mfcc.29,mfcc.30,mfcc.31,mfcc.32,mfcc.33,mfcc.34,mfcc.35,mfcc.36,mfcc.37,mfcc.38,mfcc.39,mfcc.40,mfcc.41,mfcc.4

### Merge Tracks and Features into one dataset

In [8]:
#df = pd.merge(tracks, features, on=['track_id', 'track_id'])
df = tracks

In [9]:
df.head()

,track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments.1,date_created.1,favorites.1,id.1,latitude,location,longitude,members,name,related_projects,tags.1,website,wikipedia_page,split,subset,bit_rate,comments.2,composer,date_created.2,date_recorded,duration,favorites.2,genre_top,genres,genres_all,information.1,interest,language_code,license,listens.1,lyricist,number,publisher,tags.2,title.1
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:12,2008-11-26 00:00:00,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
1,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,medium,256000,0,NaN,2008-11-26 01:48:14,2008-11-26 00:00:00,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
2,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:20,2008-11-26 00:00:00,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
3,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,[],Constant Hitmaker,2,Album,NaN,NaN,"Mexican Summer, Richie Records, Woodsist, Skul...","<p><span style=""font-family:Verdana, Geneva, A...",3,2008-11-26 01:42:55,74,6,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,"['philly', 'kurt vile']",http://kurtvile.com,NaN,training,small,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26 00:00:00,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
4,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,[],Niris,13,Album,1990-01-01 00:00:00,2011-01-01 00:00:00,NaN,<p>Songs written by: Nicky Cook</p>\n<p>VOCALS...,2,2008-11-26 01:42:52,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,"['instrumentals', 'experimental pop', 'post pu...",NaN,NaN,training,large,256000,0,NaN,2008-11-26 01:48:56,2008-01-01 00:00:00,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level


In [10]:
topgenre_unique = df['genre_top'].value_counts()
df = df[df.genre_top != 0]

print(topgenre_unique)
#Country music? Not in my classifier. Hahaha
replace_these = ["Jazz", "Old-Time / Historic", "Spoken", "Soul-RnB", "Blues", "Country","Easy Listening"]
for i in replace_these:
    df = df[df.genre_top != i]

topgenre_unique = df['genre_top'].value_counts()
print(topgenre_unique)

Rock                   14182
Experimental           10608
Electronic              9372
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: genre_top, dtype: int64
Rock             14182
Experimental     10608
Electronic        9372
Hip-Hop           3552
Folk              2803
Pop               2332
Instrumental      2079
International     1389
Classical         1230
Name: genre_top, dtype: int64


### Now a bit of Feature Engineering on the merged DF


In [11]:
# Which columns are missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(25)

,Total,Percent
lyricist,104224,0.997139
publisher,103354,0.988816
information.1,102466,0.980320
composer,100985,0.966151
wikipedia_page,99239,0.949447
active_year_end,99209,0.949160
date_recorded,98911,0.946308
related_projects,91639,0.876735
associated_labels,90395,0.864834
language_code,90364,0.864537


In [12]:
# Lets take some of the data and see if we can do some tricky feature engineering.

# Here I'll use Label Encoding 
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

# df['name'] Should encode this name. Categorical variable.
#df = df.dropna(subset=['name'])
df["name_code"] = lb_make.fit_transform(df["name"])
print(df[["name", "name_code"]].head(10))

# df['type'] Should encode this name. Categorical variable.
df = df.dropna(subset=['type'])
df["type_code"] = lb_make.fit_transform(df["type"])
print(df[["type", "type_code"]].head(10))

# df['genre_top'] Should encode this name. Categorical variable.
df = df.dropna(subset=['genre_top'])
df["genre_top_code"] = lb_make.fit_transform(df["genre_top"])
print(df[["genre_top", "genre_top_code"]].head(10))


         name  name_code
0        AWOL        297
1        AWOL        297
2        AWOL        297
3   Kurt Vile       7099
4  Nicky Cook       9159
5  Nicky Cook       9159
6  Nicky Cook       9159
7  Nicky Cook       9159
8  Nicky Cook       9159
9        AWOL        297
    type  type_code
0  Album          0
1  Album          0
2  Album          0
3  Album          0
4  Album          0
5  Album          0
6  Album          0
7  Album          0
8  Album          0
9  Album          0
       genre_top  genre_top_code
0        Hip-Hop               4
1        Hip-Hop               4
2        Hip-Hop               4
3            Pop               7
9        Hip-Hop               4
10          Rock               8
11          Rock               8
12  Experimental               2
13  Experimental               2
14          Folk               3


In [13]:
df.shape

(45682, 56)

In [14]:
# Drop all remaining columns missing data. 
nan_columns = df.columns[df.isna().any()].tolist()
df = df.drop(columns=nan_columns)
df.head(1)

# Make sure we didn't miss any.
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(4)

,Total,Percent
genre_top_code,0,0.0
type_code,0,0.0
comments,0,0.0
date_created,0,0.0


In [15]:
# All my rows are dropped that will be dropped. Now I'll make my key lists for 
df_key= df[["genre_top", "genre_top_code","type", "type_code","name", "name_code"]]

# Lets drop all the non-numerical columns now. 
data = df._get_numeric_data()
data.shape

(45682, 19)

### Test/Train Split and Prepare Some Balanced Samples. 

In [16]:
#!pip install imblearn
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import (RandomUnderSampler)

from collections import Counter
from imblearn.under_sampling import RandomUnderSampler # doctest: +NORMALIZE_WHITESPACE

# Define my X & Y
y=data["genre_top_code"]
X=data.drop(columns=['genre_top_code', 'track_id'])

# Going to try a standard scaling my X for fast converge with SAG / SAGA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# Here is where I split the model into test/train sets. 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Undersample into balanced group. 
print('Original dataset shape %s' % Counter(y_train))
rus = RandomUnderSampler(sampling_strategy='not minority',replacement=True, random_state=42)
X_RandomUS, y_RandomUS = rus.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_RandomUS))

Original dataset shape Counter({8: 11009, 2: 8233, 1: 7081, 4: 2754, 3: 2066, 7: 1821, 5: 1584, 6: 1049, 0: 948})
Resampled dataset shape Counter({0: 948, 1: 948, 2: 948, 3: 948, 4: 948, 5: 948, 6: 948, 7: 948, 8: 948})


### Define my success metrics

In [17]:
# Lets define a performance evaluation function.
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

### Now for some Logistic Regression - Original Data First

In [18]:
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV

# Logistic Regression w/ minimum random sampling of the Genre
logreg = LogisticRegression(max_iter=1000,
                            n_jobs=-1, 
                            verbose=1,
                            random_state=42)

In [19]:
# Fit with the Original Data
logreg.fit(X_train, y_train)

# lets check the performance w/ Undersampling
y_predicted = logreg.predict(X_test)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted)
trials = []
trials.append(["Trial","accuracy","precision","recall","f1"])
trial = "Base"
trials.append([trial, accuracy, precision, recall, f1])
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

[LibLinear]accuracy = 0.454, precision = 0.441, recall = 0.454, f1 = 0.398


In [20]:
# Try a basic logistic regression
# Logistic Regression w/ minimum random sampling of the Genre
logreg = LogisticRegression(verbose=1,
                            max_iter=10000,
                            random_state=42)

# Fit with the Original Data
logreg.fit(X_train, y_train)

# lets check the performance on the unbalanced data set. 
y_predicted = logreg.predict(X_test)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted)
trial = "BasicBase"
trials.append([trial, accuracy, precision, recall, f1])
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

[LibLinear]accuracy = 0.454, precision = 0.441, recall = 0.454, f1 = 0.398


### Now I attempt with the differently sampled data. 

In [21]:
# Fit with the undersampled data
logreg.fit(X_RandomUS, y_RandomUS)

# lets check the performance w/ Undersampling
y_predicted = logreg.predict(X_test)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted)
trial = 'RandomUnderSampler'
trials.append([trial, accuracy, precision, recall, f1])
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

[LibLinear]accuracy = 0.327, precision = 0.440, recall = 0.327, f1 = 0.350


In [22]:
# Lets try 2 Kinds of oversampling. 
from imblearn.over_sampling import SMOTE, ADASYN

# First SMOTE
X_overSMOTE, y_overSMOTE = SMOTE().fit_resample(X_train, y_train)
print(f"overSMOTED: {sorted(Counter(y_overSMOTE).items())}\n")

# Then ADASYN
X_overADASYN, y_overADASYN = ADASYN().fit_resample(X_train, y_train)
print(f"overADASYN: {sorted(Counter(y_overADASYN).items())}\n")


overSMOTED: [(0, 11009), (1, 11009), (2, 11009), (3, 11009), (4, 11009), (5, 11009), (6, 11009), (7, 11009), (8, 11009)]

overADASYN: [(0, 10997), (1, 10411), (2, 10894), (3, 10954), (4, 10875), (5, 11129), (6, 10981), (7, 10794), (8, 11009)]



In [23]:
# Checking how SMOTE oversampling performs. 
logreg = LogisticRegression(n_jobs=-1, 
                            verbose=1,
                            max_iter=1000,
                            random_state=42)

logreg.fit(X_overSMOTE, y_overSMOTE)

# lets check the performance w/ Oversampling 
y_predicted = logreg.predict(X_test)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted)
trial = 'SMOTE'
trials.append([trial, accuracy, precision, recall, f1])
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

[LibLinear]accuracy = 0.330, precision = 0.455, recall = 0.330, f1 = 0.348


In [24]:
# Checking how ADASYN oversampling performs. 
logreg = LogisticRegression(n_jobs=-1, 
                            verbose=1,
                            max_iter=1000,
                            random_state=42)

logreg.fit(X_overADASYN, y_overADASYN)

# lets check the performance w/ Oversampling 
y_predicted = logreg.predict(X_test)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted)
trial = 'ADASYN'
trials.append([trial, accuracy, precision, recall, f1])
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

[LibLinear]accuracy = 0.315, precision = 0.434, recall = 0.315, f1 = 0.326


### Lets try Recursive Feature Elimination to see what features are most important. 

In [26]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Lets see if we can find the ideal features.
logreg = LogisticRegression(n_jobs=-1, random_state=42, verbose=1, max_iter=100, warm_start=True)

rfe = RFE(logreg, n_features_to_select=1, verbose=1)
rfe = rfe.fit(X_test, y_test)

print(rfe.support_)
print(rfe.ranking_)

Fitting estimator with 17 features.
[LibLinear]Fitting estimator with 16 features.
[LibLinear]Fitting estimator with 15 features.
[LibLinear]Fitting estimator with 14 features.
[LibLinear]Fitting estimator with 13 features.
[LibLinear]Fitting estimator with 12 features.
[LibLinear]Fitting estimator with 11 features.
[LibLinear]Fitting estimator with 10 features.
[LibLinear]Fitting estimator with 9 features.
[LibLinear]Fitting estimator with 8 features.
[LibLinear]Fitting estimator with 7 features.
[LibLinear]Fitting estimator with 6 features.
[LibLinear]Fitting estimator with 5 features.
[LibLinear]Fitting estimator with 4 features.
[LibLinear]Fitting estimator with 3 features.
[LibLinear]Fitting estimator with 2 features.
[LibLinear][LibLinear][False False False False False False False False False False False  True
 False False False False False]
[16 14 11  3  2  9  7 10 12 17  8  1  4  5 13 15  6]


In [28]:
names = data.drop(columns=['genre_top_code','track_id'])
feature_names = list(names.columns)

print ("Features sorted by their rank:")
print (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), feature_names)))

Features sorted by their rank:
[(1, 'favorites.2'), (2, 'tracks'), (3, 'listens'), (4, 'interest'), (5, 'listens.1'), (6, 'type_code'), (7, 'favorites.1'), (8, 'duration'), (9, 'comments.1'), (10, 'id.1'), (11, 'id'), (12, 'bit_rate'), (13, 'number'), (14, 'favorites'), (15, 'name_code'), (16, 'comments'), (17, 'comments.2')]


In [79]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import (LinearRegression, Ridge, 
                                  Lasso, RandomizedLasso)
from sklearn.feature_selection import RFE,RFECV,f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics
import numpy as np
#!pip install minepy
from minepy import MINE
sorted(sklearn.metrics.SCORERS.keys()) 

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [90]:
# Scoring types to consider
# ‘neg_mean_squared_error’, 'accuracy', 'precision', 'recall', ‘neg_log_loss’

 
ranks = {}
names = feature_names

Y= y_test
X= X_test

print(Y.shape)
print(X.shape)

logreg.fit(X, Y)

def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks ))

print (np.abs(logreg.coef_).ndim)

logreg = LogisticRegression(n_jobs=-1, random_state=42, verbose=1, max_iter=100, warm_start=True)
#rfecv.grid_scores_
rfecv = RFECV(logreg, min_features_to_select=1, step=1, cv=StratifiedKFold(2),
              scoring='r2',verbose=1)
rfecv.fit(X,Y)

#‘neg_mean_squared_error’, 'accuracy', 'precision', 'recall', ‘neg_log_loss’'f1_weighted'

#ranks["RFECV"] = rank_to_dict(([rfecv.grid_scores_]), names, order=-1)
#ranks["RFE"] = rank_to_dict((float(i) for i in rfe.ranking_), names, order=-1)
#rfe.ranking_
print(rfecv.ranking_)
print(rfecv.grid_scores_)
#print(ranking = [float(i) for i in ranking])
#print(map(float, ranking))
#order=-1
#print(np.array(float(i) for i in ranking))
#print(order*np.array([float(i) for i in rfe.ranking_]))
ranks["Stability"] = rank_to_dict(np.abs([1-i for i in rfecv.grid_scores_]), names)
print (ranks)

(9137,)
(9137, 17)
[LibLinear]2
Fitting estimator with 17 features.
[LibLinear]Fitting estimator with 16 features.
[LibLinear]Fitting estimator with 15 features.
[LibLinear]Fitting estimator with 14 features.
[LibLinear]Fitting estimator with 13 features.
[LibLinear]Fitting estimator with 12 features.
[LibLinear]Fitting estimator with 11 features.
[LibLinear]Fitting estimator with 10 features.
[LibLinear]Fitting estimator with 9 features.
[LibLinear]Fitting estimator with 8 features.
[LibLinear]Fitting estimator with 7 features.
[LibLinear]Fitting estimator with 6 features.
[LibLinear]Fitting estimator with 5 features.
[LibLinear]Fitting estimator with 4 features.
[LibLinear]Fitting estimator with 3 features.
[LibLinear]Fitting estimator with 2 features.
[LibLinear][LibLinear]Fitting estimator with 17 features.
[LibLinear]Fitting estimator with 16 features.
[LibLinear]Fitting estimator with 15 features.
[LibLinear]Fitting estimator with 14 features.
[LibLinear]Fitting estimator with 13

In [99]:
#https://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import (LinearRegression, Ridge, 
                                  Lasso)
from sklearn.feature_selection import RFE,RFECV,f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
#!pip install minepy
#from minepy import MINE
 
np.random.seed(0)
 
size = 750
 
Y = y_test
X = X_test
 
names = feature_names
 
ranks = {}
 
def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks ))
 
lr = LinearRegression(normalize=True)
lr.fit(X, Y)
ranks["Linear reg"] = rank_to_dict(np.abs(lr.coef_), names)

rf = RandomForestRegressor()
rf.fit(X,Y)
ranks["RF"] = rank_to_dict(rf.feature_importances_, names)

f, pval  = f_regression(X, Y, center=True)
ranks["Corr.(p_value)"] = rank_to_dict(f, names)

ridge = Ridge(alpha=7)
ridge.fit(X, Y)
ranks["Ridge"] = rank_to_dict(np.abs(ridge.coef_), names)

lasso = Lasso(alpha=.05)
lasso.fit(X, Y)
ranks["Lasso"] = rank_to_dict(np.abs(lasso.coef_), names)

# Depreciated out of scikit learn
#rlasso = RandomizedLasso(alpha=0.04)
#rlasso.fit(X, Y)
#ranks["Stability"] = rank_to_dict(np.abs(rlasso.scores_), names)

#stop the search when 5 features are left (they will get equal scores)
rfe = RFE(logreg, n_features_to_select=5)
rfe.fit(X,Y)
ranks["RFE"] = rank_to_dict(([float(i) for i in rfe.ranking_]), names, order=-1)

#mine = MINE()
#mic_scores = []
#for i in range(X.shape[1]):
#    mine.compute_score(X[:,i], Y)
#    m = mine.mic()
#    mic_scores.append(m)
#ranks["MIC"] = rank_to_dict(mic_scores, names) 

r = {}
for name in names: r[name] = round(np.mean([ranks[method][name]for method in ranks.keys()]), 2)
ranks["Mean"] = r
    
#rfecv.grid_scores_- Accuracy Contribution
rfecv = RFECV(logreg, min_features_to_select=1, step=1, cv=StratifiedKFold(2),
              scoring='accuracy',verbose=1)
rfecv.fit(X,Y)
ranks["Accuracy"] = rank_to_dict(np.abs([1-i for i in rfecv.grid_scores_]), names)

#rfecv.grid_scores_ - Precision Contribution
rfecv = RFECV(logreg, min_features_to_select=1, step=1, cv=StratifiedKFold(2),
              scoring='precision_weighted',verbose=1)
rfecv.fit(X,Y)
ranks["Precision"] = rank_to_dict(np.abs([1-i for i in rfecv.grid_scores_]), names)

#rfecv.grid_scores_ - Recall Contribution
rfecv = RFECV(logreg, min_features_to_select=1, step=1, cv=StratifiedKFold(2),
              scoring='recall_weighted',verbose=1)
rfecv.fit(X,Y)
ranks["Recall"] = rank_to_dict(np.abs([1-i for i in rfecv.grid_scores_]), names)

#rfecv.grid_scores_ - F1 Contribution
rfecv = RFECV(logreg, min_features_to_select=1, step=1, cv=StratifiedKFold(2),
              scoring='f1_weighted',verbose=1)
rfecv.fit(X,Y)
ranks["f1"] = rank_to_dict(np.abs([1-i for i in rfecv.grid_scores_]), names)


methods = list(ranks.keys())
#methods.append("Mean")

df_tabs = pd.DataFrame.from_dict(ranks)
df_tabs = df_tabs[methods]
df_tabs

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Fitting estimator with 17 features.
[LibLinear]Fitting estimator with 16 features.
[LibLinear]Fitting estimator with 15 features.
[LibLinear]Fitting estimator with 14 features.
[LibLinear]Fitting estimator with 13 features.
[LibLinear]Fitting estimator with 12 features.
[LibLinear]Fitting estimator with 11 features.
[LibLinear]Fitting estimator with 10 features.
[LibLinear]Fitting estimator with 9 features.
[LibLinear]Fitting estimator with 8 features.
[LibLinear]Fitting estimator with 7 features.
[LibLinear]Fitting estimator with 6 features.
[LibLinear]Fitting estimator with 5 features.
[LibLinear]Fitting estimator with 4 features.
[LibLinear]Fitting estimator with 3 features.
[LibLinear]Fitting estimator with 2 features.
[LibLinear][LibLinear]Fitting estimator with 17 features.
[LibLinear]Fitting estimator with 16 features.
[LibLinear]Fitting 

,Linear reg,RF,Corr.(p_value),Ridge,Lasso,RFE,Mean,Accuracy,Precision,Recall,f1
bit_rate,0.07,0.42,0.07,0.08,0.14,0.42,0.20,0.09,0.13,0.09,0.11
comments,0.01,0.09,0.05,0.01,0.00,0.08,0.04,1.00,1.00,1.00,1.00
comments.1,0.00,0.16,0.04,0.00,0.00,0.67,0.15,0.28,0.30,0.28,0.26
comments.2,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.09,0.04,0.09,0.10
duration,0.20,0.85,0.19,0.23,0.48,0.75,0.45,0.10,0.06,0.10,0.10
favorites,0.08,0.23,0.10,0.09,0.18,0.25,0.16,0.69,0.56,0.69,0.58
favorites.1,0.00,0.57,0.03,0.00,0.00,0.83,0.24,0.22,0.28,0.22,0.19
favorites.2,0.02,0.18,0.05,0.02,0.00,1.00,0.21,0.04,0.04,0.04,0.05
id,0.12,0.89,0.03,0.14,0.00,0.50,0.28,0.63,0.51,0.63,0.52
id.1,0.17,0.94,0.00,0.19,0.11,0.58,0.33,0.09,0.14,0.09,0.11


### Add in the Features.csv

In [ ]:
# Merge Df's 
df = pd.merge(tracks, features, on=['track_id', 'track_id'])
print(df.shape)

# Fix Genres list. 
topgenre_unique = df['genre_top'].value_counts()
df = df[df.genre_top != 0]

# Country music? Not in my classifier. Hahaha
replace_these = ["Jazz", "Old-Time / Historic", "Spoken", "Soul-RnB", "Blues", "Country","Easy Listening"]
for i in replace_these:
    df = df[df.genre_top != i]

# Lets take some of the data and see if we can do some tricky feature engineering.
# Here I'll use Label Encoding 
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

# df['name'] Should encode this name. Categorical variable.
#df = df.dropna(subset=['name'])
df["name_code"] = lb_make.fit_transform(df["name"])

# df['type'] Should encode this name. Categorical variable.
df = df.dropna(subset=['type'])
df["type_code"] = lb_make.fit_transform(df["type"])

# df['genre_top'] Should encode this name. Categorical variable.
df = df.dropna(subset=['genre_top'])
df["genre_top_code"] = lb_make.fit_transform(df["genre_top"])

# Drop all remaining columns missing data. 
nan_columns = df.columns[df.isna().any()].tolist()
df = df.drop(columns=nan_columns)
df.head(1)

# All my rows are dropped that will be dropped. Now I'll make my key lists for 
df_key= df[["genre_top", "genre_top_code","type", "type_code","name", "name_code"]]

# Lets drop all the non-numerical columns now. 
data = df._get_numeric_data()
print(data.shape)

# Define my X & Y
y=data["genre_top_code"]
X=data.drop(columns=['genre_top_code'])

# Going to try a standard scaling my X for fast converge with SAG / SAGA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# Here is where I split the model into test/train sets. 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Undersample into balanced group. 
print('Original dataset shape %s' % Counter(y_train))
rus = RandomUnderSampler(sampling_strategy='not minority',replacement=True, random_state=42)
X_RandomUS, y_RandomUS = rus.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_RandomUS))

In [ ]:
logreg = LogisticRegression(class_weight ='balanced',
                            multi_class='auto', 
                            solver='lbfgs',
                            n_jobs=-1, 
                            verbose=1,
                            max_iter=500,
                            random_state=42)

## Guidance for the Assignment




This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.